In [ ]:
"""
Created on Fri Jul 09 09:39 2021

This script is to transform the mean profiles to 2D fields to accelerate the tuning process

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
from scipy import stats
from dask import delayed

import distributed
import glob

In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=16,dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

READ IN DATA

In [ ]:
nemo_run = 'OPM006'

In [ ]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'
plot_path = '/bettik/burgardc/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 300

Files for param

In [ ]:
file_other = xr.open_mfdataset(inputpath_data+'corrected_draft_bathy_isf.nc')
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim).chunk(chunks={'x': chunk_size, 'y': chunk_size})
file_conc = xr.open_mfdataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim).chunk(chunks={'x': chunk_size, 'y': chunk_size})
file_isf_conc = file_conc_cut['isfdraft_conc']


In [ ]:
file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf).chunk(chunks={'x': chunk_size, 'y': chunk_size})


file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc', chunks={'Nisf': 1, 'time': 5})
file_TS = file_TS_orig.sel(Nisf=large_isf)

In [ ]:
param_var_of_int = file_isf[['ISF_mask', 'latitude', 'longitude', 'front_bot_depth_max']]
plume_charac = xr.open_mfdataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc', chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
file_TS

First 2D fields of thermal forcing

In [ ]:
ice_draft_isf = ice_draft_pos.where(param_var_of_int['ISF_mask'] == file_isf.Nisf).chunk(chunks={'Nisf': 1})


In [ ]:
filled_TS

In [ ]:
filled_TS = file_TS.ffill(dim='depth').chunk({'time': 5}) #, 'profile_domain': 1})


for ref in ['isf_draft']:#,'bottom_front']:
    
    print(ref)
    # DOES NOT WORK YET
    if ref == 'bottom_front':
        # Entering temperature and salinity profiles
        n = 0
        for kisf in file_isf.Nisf:
            depth_of_int_kisf = param_var_of_int['front_bot_depth_max'].sel(Nisf=kisf).where(file_isf['ISF_mask']==kisf)
            if n == 0:
                depth_of_int = depth_of_int_kisf.squeeze().drop('Nisf')
            else:
                depth_of_int = depth_of_int.combine_first(depth_of_int_kisf).squeeze().drop('Nisf')
            n = n+1
            
    elif ref == 'isf_draft':
        # ice draft depth or deepest entrance depth
        depth_of_int = ice_draft_isf.where(ice_draft_isf<param_var_of_int['front_bot_depth_max'], 
                                           param_var_of_int['front_bot_depth_max']).chunk(chunks={'Nisf':1})
        depth_of_int = depth_of_int.where(file_isf['ISF_mask'] == file_isf.Nisf, 0).chunk({'Nisf': 1})
        #depth_of_int = depth_of_int.where(file_isf['ISF_mask'] > 1).sum('Nisf')
        #depth_of_int = depth_of_int.where(depth_of_int>0)
        
    elif ref == 'GL_depth_cavity':
        # deepest GL point
        depth_of_int = -1*plume_charac['zGL'].sel(option='simple').where(file_isf['ISF_mask']==file_isf.Nisf,0).chunk({'Nisf': 1})
        #depth_of_int = depth_of_int.where(depth_of_int>0)
    elif ref == 'GL_depth_lazero':
        # depth from Lazero
        depth_of_int = -1*plume_charac['zGL'].sel(option='lazero').where(file_isf['ISF_mask']==file_isf.Nisf,0).chunk({'Nisf': 1})
        #depth_of_int = depth_of_int.where(depth_of_int>0)
        
    print('here1')
    T_isf = filled_TS['theta_ocean'].interp({'depth': depth_of_int}).drop('depth')
    T_isf = T_isf.where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    T_isf = T_isf.to_dataset(name='theta_in')
    print('here2')
    T_isf['salinity_in'] = filled_TS['salinity_ocean'].interp({'depth': depth_of_int}).drop('depth')
    T_isf['salinity_in'] = T_isf['salinity_in'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    print('here3')
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] > 1).sum('Nisf')
    depth_of_int = depth_of_int.where(depth_of_int>0)
    T_isf['freezing_T'] = meltf.freezing_temperature(T_isf['salinity_in'], -1*depth_of_int)
    print('here4')
    T_isf['thermal_forcing'] = T_isf['theta_in'] - T_isf['freezing_T']
    T_isf['depth_of_int'] = depth_of_int
    
    # project it on 2D
    #T_isf = T_isf.sum('Nisf')
    T_isf.to_netcdf(inputpath_profiles+'T_S_2D_fields_'+ref+'_oneFRIS.nc','w')
    del T_isf
    del depth_of_int